In [12]:
from dotenv import dotenv_values
import os

config = dotenv_values(".env.secret")  # load sensitive variables
print(config.keys())

for c, v in config.items():
    os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [13]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
import torch

torch.cuda.is_available()

True

In [15]:
import torch

torch.cuda.device_count()

1

In [16]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [17]:
import os

max(1, os.cpu_count() -1)

11

In [18]:
from src.data import HousePricingDataModule

batch_size = 256

data_module = HousePricingDataModule(batch_size=batch_size)
data_module.prepare_data()

for stage in ["fit"]:
    data_module.setup(stage=stage)

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)
[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 981
[INFO]: Validation dataset size: 173


In [19]:
import os
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
import wandb
from src.model import NeuralNetwork

project_name = os.environ["WANDB_NAME"]
wandb_logger = WandbLogger(project=project_name)

train_dataloader = data_module.train_dataloader()
val_dataloader = data_module.val_dataloader()

in_features = data_module.in_features()

model = NeuralNetwork(in_features)

max_epochs = 500

H = {
    "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
    "in_features": in_features,
    "max_epochs": max_epochs
}

for c, v in H.items():
    wandb_logger.experiment.config[c] = v

trainer = Trainer(
    accelerator=H["accelerator"],
    max_epochs=H["max_epochs"],
    logger=wandb_logger,
    log_every_n_steps=1)
trainer.fit(model, datamodule=data_module)


[INFO]: Train dataloader size: 4
[INFO]: Validation dataloader size: 1
[INFO]: Input size: 244


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 215 K 
------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.864     Total estimated model params size (MB)


[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 981
[INFO]: Validation dataset size: 173
Sanity Checking: |          | 0/? [00:00<?, ?it/s][INFO]: Validation dataloader size: 1
                                                                            

/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Train dataloader size: 4
Epoch 499: 100%|██████████| 4/4 [00:00<00:00, 35.13it/s, v_num=dhce]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 4/4 [00:00<00:00, 31.41it/s, v_num=dhce]


In [20]:
trainer.test(model, datamodule=data_module)

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 2
Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 183.52it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss                623093824.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 623093824.0}]

In [21]:
trainer.predict(model, datamodule=data_module)

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up predict dataset/s
[INFO]: Predict dataloader size: 6
Predicting DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 99.49it/s] 


[tensor([[100534.5781],
         [137431.3281],
         [177545.6094],
         [184588.4375],
         [186280.1719],
         [163127.3281],
         [172733.9844],
         [148176.5781],
         [191457.9844],
         [105468.8125],
         [148269.6406],
         [ 85805.6250],
         [ 85284.5547],
         [131375.7500],
         [101238.7500],
         [357325.7812],
         [235684.3281],
         [292270.5938],
         [302105.6250],
         [462479.4062],
         [295478.9062],
         [187565.7031],
         [167656.2500],
         [157388.4219],
         [163867.9531],
         [180955.0156],
         [316074.5938],
         [209186.1094],
         [192389.7656],
         [249059.5469],
         [184183.9688],
         [ 93728.6953],
         [179874.6562],
         [314697.3125],
         [274676.9062],
         [240525.9688],
         [167366.4688],
         [164404.5625],
         [149146.9531],
         [141741.0000],
         [190768.3906],
         [123276

In [22]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,500
train_loss,411055424.0
trainer/global_step,2000
val_loss,623093824.0
